In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
pd.options.display.max_rows = 6
import pickle

In [2]:
data = pd.read_csv('Absenteeism-data.csv')
raw_data = data.copy()

In [3]:
print(data.keys())
data.head()

Index(['ID', 'Reason for Absence', 'Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index',
       'Education', 'Children', 'Pets', 'Absenteeism Time in Hours'],
      dtype='object')


,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [4]:
data = data.drop(['ID'],axis=1)

In [5]:
print('num. of unique values = ',len(data['Reason for Absence'].unique()))

num. of unique values =  28


In [6]:
reasons_columns = pd.get_dummies(data['Reason for Absence'],drop_first=True)

In [7]:
reasons_columns

,1,2,3,4,5,6,7,8,9,10,...,18,19,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
data = data.drop(['Reason for Absence'],axis=1)

In [9]:
grouped = pd.DataFrame()
grouped['reason_1'] = reasons_columns.loc[:,1:14].max(axis=1)
grouped['reason_2']  = reasons_columns.loc[:,15:17].max(axis=1)
grouped['reason_3']  = reasons_columns.loc[:,18:21].max(axis=1)
grouped['reason_4']  = reasons_columns.loc[:,22:].max(axis=1)
grouped

,reason_1,reason_2,reason_3,reason_4
0,0,0,0,1
1,0,0,0,0
2,0,0,0,1
...,...,...,...,...
697,1,0,0,0
698,0,0,0,1
699,0,0,0,1


In [10]:
data['Date'] = pd.to_datetime(data['Date'],format = '%d/%m/%Y')

In [11]:
#saving the month & day in the week value in seperate column
data['month'] = data.apply(lambda row: row['Date'].month, axis=1)
data['day_of_the_week'] = data.apply(lambda row: row['Date'].weekday() ,axis=1)

In [12]:
grouped['Education'] = data['Education'].map({1:0,2:1,3:1,4:1})

In [13]:
grouped['Excessive_absenteeism'] = np.where(data['Absenteeism Time in Hours']> data['Absenteeism Time in Hours'].median(),1,0)

In [14]:
#checking balancing
grouped['Excessive_absenteeism'].sum()/grouped['Excessive_absenteeism'].shape[0]

0.45571428571428574

In [15]:
data = data.drop(['Education','Date','Absenteeism Time in Hours'],axis=1)

In [16]:
columns = data.columns
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)
with open('scaler','wb') as file:
    pickle.dump(scaler,file)

In [17]:
data = pd.DataFrame(data,columns = columns)

In [18]:
data = pd.concat([grouped,data],axis=1)
data

,reason_1,reason_2,reason_3,reason_4,Education,Excessive_absenteeism,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Children,Pets,month,day_of_the_week
0,0,0,0,1,0,1,1.005844,0.412816,-0.536062,-0.806331,0.767431,0.880469,0.268487,0.182726,-0.683704
1,0,0,0,0,0,0,-1.574681,-1.141882,2.130803,-0.806331,1.002633,-0.019280,-0.589690,0.182726,-0.683704
2,0,0,0,1,0,0,-0.654143,1.426749,0.248310,-0.806331,1.002633,-0.919030,-0.589690,0.182726,-0.007725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,1,0,0,0,1,1,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,-0.919030,-0.589690,-0.388293,0.668253
698,0,0,0,1,1,0,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,-0.919030,-0.589690,-0.388293,0.668253
699,0,0,0,1,0,0,1.036026,0.074838,0.562059,-0.853789,-0.408580,-0.019280,0.268487,-0.388293,0.668253


In [19]:
data = data[[ 'reason_1', 'reason_2', 'reason_3', 'reason_4','month','day_of_the_week', 'Transportation Expense', 'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 'Pets', 'Excessive_absenteeism']]

In [20]:
data.to_csv('./Absenteeism_data_preprocessed.csv',index=False)